In [34]:
import mlflow
import onnx
import numpy as np
import time
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import xgboost as xgb
from onnxmltools.convert.common.data_types import FloatTensorType
import onnxruntime as rt

In [16]:
os.environ["AWS_ACCESS_KEY_ID"] = "mlflow"
os.environ["AWS_SECRET_ACCESS_KEY"] = "password"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://localhost:9000"

In [17]:
mlflow.set_tracking_uri('http://localhost:9909')
experiment_name = "credit_scoring"
mlflow.set_experiment(experiment_name)
mlflow.autolog()

2024/12/03 15:50:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/12/03 15:50:52 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.2, but the installed version is 2.1.3. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2024/12/03 15:50:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [36]:
from copy import deepcopy
train_data = pd.read_csv('/Users/mark/Documents/work/mlflow-tutorial/data/train.csv')
test_data = pd.read_csv('/Users/mark/Documents/work/mlflow-tutorial/data/test.csv')

X = train_data.drop('loan_status', axis=1)
y = train_data['loan_status']
def preprocess_data(df):
    data = deepcopy(df)
    categorical_features = data.select_dtypes(include=['object']).columns
    for feature in categorical_features:
        le = LabelEncoder()
        data[feature] = le.fit_transform(data[feature].astype(str))

    scaler = StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    return data

X = preprocess_data(X)

X_test = preprocess_data(test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
model_name = "raw_xgb"  
model_version = 1  

model = mlflow.xgboost.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

n_features = len(model.feature_names_in_)
old_feature_names = model.feature_names_in_
feature_names = [f'f{i}' for i in range(n_features)]
model._Booster.feature_names = feature_names

initial_type = [('float_input', FloatTensorType([None, n_features]))]

onx = onnxmltools.convert_xgboost(model, initial_types=initial_type)

with open("raw_xgb.onnx", "wb") as f:
    f.write(onx.SerializeToString())

X_test.columns = feature_names


sess_options = rt.SessionOptions()
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL
sess_options.intra_op_num_threads = -1


sess = rt.InferenceSession(
    "raw_xgb.onnx",
    sess_options=sess_options,
    providers=["CPUExecutionProvider"]
)
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
X_val_array = X_test.values
X_val_dict = {input_name: X_val_array.astype(np.float32)}

start_time_onnx = time.time()
pred_onnx = sess.run([label_name], X_val_dict)[0]
onnx_time = time.time() - start_time_onnx

start_time_xgb = time.time()
pred_xgb = model.predict(X_test)
xgb_time = time.time() - start_time_xgb

print("Current provider:", rt.get_device())
print("Predictions are equal:", np.allclose(pred_xgb, pred_onnx))
print(f"ONNX prediction time: {onnx_time:.4f} seconds")
print(f"XGBoost prediction time: {xgb_time:.4f} seconds")
print(f"ONNX is {xgb_time/onnx_time:.2f}x faster than XGBoost")

Current provider: CPU
Predictions are equal: True
ONNX prediction time: 0.0218 seconds
XGBoost prediction time: 0.0114 seconds
ONNX is 0.52x faster than XGBoost
